In [ ]:
# The Central Index Key (CIK) is used on the SEC's computer systems to identify corporations and individual people 
# who have filed disclosure with the SEC.

In [11]:
# Requirements
import pandas as pd
import csv
import requests
import json
import time
import os

folder_name = 'Data'
os.makedirs(f'{folder_name}',exist_ok=True)
headers = {'User-Agent': 'Temp Company LLC tempcompanyllc@gmail.com'}

In [3]:
# Get Recent Filings by providing CIK

def main(cik):
    url = f'https://data.sec.gov/submissions/CIK{cik}.json'
    r = requests.get(url, headers=headers)
    df = pd.DataFrame.from_dict(r.json()['filings']['recent'])
    df.to_csv(f'{folder_name}/{cik}-RecentFilings.csv', index=False)

main('0001018724')

In [4]:
# Get Recent Filings by provindg a list of CIKs

pd.read_csv(f'{folder_name}/ListofCiks.csv', dtype='string')

def main(filename):
    listofciks = pd.read_csv(f'{folder_name}/{filename}', dtype='string')

    start_time = time.time()
    for cik in listofciks['CIK']:
        name = listofciks[listofciks['CIK'] == cik]['Name'].iloc[0].replace('/','')
        url = f'https://data.sec.gov/submissions/CIK{cik}.json'
        r = requests.get(url, headers=headers)
        df = pd.DataFrame.from_dict(r.json()['filings']['recent'])
        df.to_csv(f'{folder_name}/{name} - RecentFilings.csv', index=False)

        end_time = time.time()
        if end_time - start_time < 1:
            time.sleep(0.1)
        else:
            pass

main('ListofCIKs.csv')

In [12]:
# Get Recent Filings for all Companies with a Ticker Symbol

def tickers():
    url = 'https://www.sec.gov/files/company_tickers.json'
    r = requests.get(url=url, headers=headers)
    data = r.json()
    df = pd.DataFrame(data)
    df = df.T
    df.to_csv(f'{folder_name}/CompanyTickers.csv', index=False)

tickers()

def main():
    CompanyTickers = pd.read_csv(f'{folder_name}/CompanyTickers.csv', dtype='string')

    start_time = time.time()
    for cik in CompanyTickers['cik_str']:
        name = CompanyTickers[CompanyTickers['cik_str'] == cik]['title'].iloc[0].replace('/','')

        if len(cik) < 10:
                add_zeros = 10 - len(cik)
                cik = add_zeros * "0" + cik
        else:
            pass
        
        url = f'https://data.sec.gov/submissions/CIK{cik}.json'
        r = requests.get(url, headers=headers)
        df = pd.DataFrame.from_dict(r.json()['filings']['recent'])
        df.to_csv(f'{folder_name}/{name} - RecentFilings.csv', index=False)

        end_time = time.time()
        if end_time - start_time < 1:
            time.sleep(0.1)
        else:
            pass

main()